In [123]:
import cv2 
import pytesseract as pt
import numpy as np
from PIL import Image
from pytesseract import Output
import re


In [124]:

image= cv2.imread('download.jpg')



In [125]:
#defining all preprocessing and we will use them according to our image
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)



#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED) 

In [126]:
#using preprocessing which is needed for image
gray = get_grayscale(image)
thresh = thresholding(gray)
Image.fromarray(thresh).show()
canny = canny(gray)


In [127]:

pt.pytesseract.tesseract_cmd=r"D:\tesseract\Tesseract.exe"
custom_config = r'--oem 3 --psm 6 outputbase digits'
#we will bound box around all the values which we want to add
d = pt.image_to_data(gray, output_type=Output.DICT)
keys=list(d.keys())

data_pattern='^[0-9]+$'

n_boxes = len(d['text'])
for i in range(n_boxes):
    if int(d['conf'][i]) > 60:
        if(re.match(data_pattern,d['text'][i])):
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            img = cv2.rectangle(thresh, (x, y), (x + w, y + h), (0, 255, 0), 2)

Image.fromarray(img).show()



In [128]:
#Adding all the values by first storing them on list and then adding them
out=list(pt.image_to_string(gray, config=custom_config).split("\n"))
out1=[]
for i in out:
    i=''.join(j for j in i if j in ["1","2","3","4","5","6","7","8","9","0"])
    out1.append(i)
    
while("" in out1) : 
    out1.remove("") 
print(out1)
total=0
print("The numbers are")
for i in out1:
    print(i)
    total+=int(i)
print("total: ",total)



['123', '456', '789']
The numbers are
123
456
789
total:  1368
